In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
use_cuda = True
log_interval = 512
device = torch.device('cuda' if (torch.cuda.is_available() and use_cuda) else 'cpu')
epochs = 20
lr = 1e-3
momentum = 0.5
test_batch_size = 512
batch_size = 64
save_model = False

#### A

#### Default architecture from https://github.com/pytorch/examples/blob/master/mnist/main.py

In [3]:
class Def_Net(nn.Module):
    def __init__(self):
        super(Def_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Train test functions for simplicity

In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
### Some additional code to configure dataloaders
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [6]:
model = Def_Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

if (save_model):
    torch.save(model.state_dict(),"SavedWeights/mnist_cnn_relu.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298247
Train Epoch: 1 [32768/60000 (55%)]	Loss: 1.555109

Test set: Average loss: 0.5057, Accuracy: 8741/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.431973
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.528761

Test set: Average loss: 0.2795, Accuracy: 9192/10000 (92%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.203300
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.167266

Test set: Average loss: 0.2147, Accuracy: 9378/10000 (94%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.446896
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.247596

Test set: Average loss: 0.1712, Accuracy: 9517/10000 (95%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.292055
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.192984

Test set: Average loss: 0.1506, Accuracy: 9580/10000 (96%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.169419
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.211955

Test set: Average loss: 0.1254, Accuracy: 9639/10000 (96%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.104067
Train Epoch: 7 [327

The above results correspond to the default architecture mentioned in the tutoraial link. With default arguments (mentioned in the tutorial) for the hyperparameters in cell 2, the architecture reached an accuracy of 98% in 10 epochs and a learning rate of 1e-3.

Below, all the ReLUs are replaced with sigmoids and the network is being trained again to see how it performs

In [7]:
class Sig_Net(nn.Module):
    def __init__(self):
        super(Sig_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.sigmoid(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.sigmoid(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

#### Training the sigmoid network

In [8]:
epochs = 100
model_sig = Sig_Net().to(device)
optimizer = optim.SGD(model_sig.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_sig, device, train_loader, optimizer, epoch)
    test(model_sig, device, test_loader)

if (save_model):
    torch.save(model_sig.state_dict(),"SavedWeights/mnist_cnn_sigm.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.434300


/home/grumptitan/.local/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [32768/60000 (55%)]	Loss: 2.304287

Test set: Average loss: 2.3010, Accuracy: 1135/10000 (11%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.312151
Train Epoch: 2 [32768/60000 (55%)]	Loss: 2.296524

Test set: Average loss: 2.3007, Accuracy: 1135/10000 (11%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.314402
Train Epoch: 3 [32768/60000 (55%)]	Loss: 2.327645

Test set: Average loss: 2.3004, Accuracy: 1135/10000 (11%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.301714
Train Epoch: 4 [32768/60000 (55%)]	Loss: 2.297709

Test set: Average loss: 2.2993, Accuracy: 1135/10000 (11%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 2.299953
Train Epoch: 5 [32768/60000 (55%)]	Loss: 2.297132

Test set: Average loss: 2.2994, Accuracy: 1135/10000 (11%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 2.301347
Train Epoch: 6 [32768/60000 (55%)]	Loss: 2.308126

Test set: Average loss: 2.2985, Accuracy: 1135/10000 (11%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 2.303738
Train Epoch: 7 [32768/60000 (55%)]	Loss: 2.309259

Test set: Ave

Train Epoch: 53 [32768/60000 (55%)]	Loss: 0.554777

Test set: Average loss: 0.4617, Accuracy: 8685/10000 (87%)

Train Epoch: 54 [0/60000 (0%)]	Loss: 0.420156
Train Epoch: 54 [32768/60000 (55%)]	Loss: 0.555521

Test set: Average loss: 0.4503, Accuracy: 8714/10000 (87%)

Train Epoch: 55 [0/60000 (0%)]	Loss: 0.309469
Train Epoch: 55 [32768/60000 (55%)]	Loss: 0.486180

Test set: Average loss: 0.4396, Accuracy: 8750/10000 (88%)

Train Epoch: 56 [0/60000 (0%)]	Loss: 0.388046
Train Epoch: 56 [32768/60000 (55%)]	Loss: 0.434048

Test set: Average loss: 0.4289, Accuracy: 8776/10000 (88%)

Train Epoch: 57 [0/60000 (0%)]	Loss: 0.528628
Train Epoch: 57 [32768/60000 (55%)]	Loss: 0.385626

Test set: Average loss: 0.4196, Accuracy: 8807/10000 (88%)

Train Epoch: 58 [0/60000 (0%)]	Loss: 0.443321
Train Epoch: 58 [32768/60000 (55%)]	Loss: 0.364668

Test set: Average loss: 0.4103, Accuracy: 8841/10000 (88%)

Train Epoch: 59 [0/60000 (0%)]	Loss: 0.399577
Train Epoch: 59 [32768/60000 (55%)]	Loss: 0.683918



As we expected, sigmoid activations are subpar when compared to ReLU in this case. This can be explained by the following argument.

Consider 2 deep network with just Fully connected layers. One has ReLU after all layers and one has sigmoid after all layers 

Due to the underlying architecture, the output of every node in every layer is restricted between 0 and 1 if sigmoid activation is used. Otherwise, the node outputs which are negative are suppressed when ReLU is used. Restricting output from 0 to 1 can be considered as a loss of information as withing 0 and 1 we will be dealing with numbers only upto a precision.

In case of images, since it's a convolutional operation going, we would like to have our values uncapped so that we can understand the response through the filter. Hence, ReLU clearly outperforms sigmoid. 

#### B

Dropout of probability P applied after last fully connected layer convolution

In [9]:
class DrO_Net(nn.Module):
    def __init__(self, prob):
        super(DrO_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.do1 = nn.Dropout(prob)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.do1(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [10]:
### Dropout of 0.2 from first convolution
epochs = 20
model_25 = DrO_Net(prob=0.25).to(device)
optimizer = optim.SGD(model_25.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_25, device, train_loader, optimizer, epoch)
    test(model_25, device, test_loader)

if (save_model):
    torch.save(model_25.state_dict(),"SavedWeights/mnist_cnn_dr_25.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297590
Train Epoch: 1 [32768/60000 (55%)]	Loss: 1.720791

Test set: Average loss: 0.5770, Accuracy: 8617/10000 (86%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.712266
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.359807

Test set: Average loss: 0.2894, Accuracy: 9187/10000 (92%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.253130
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.409865

Test set: Average loss: 0.2177, Accuracy: 9376/10000 (94%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.194865
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.261231

Test set: Average loss: 0.1736, Accuracy: 9509/10000 (95%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.134154
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.102249

Test set: Average loss: 0.1477, Accuracy: 9576/10000 (96%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.223577
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.182804

Test set: Average loss: 0.1259, Accuracy: 9640/10000 (96%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.054737
Train Epoch: 7 [327

In [11]:
### Dropout of 0.5 from first convolution
model_50 = DrO_Net(prob=0.5).to(device)
optimizer = optim.SGD(model_50.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_50, device, train_loader, optimizer, epoch)
    test(model_50, device, test_loader)

if (save_model):
    torch.save(model_50.state_dict(),"SavedWeights/mnist_cnn_dr_50.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.289551
Train Epoch: 1 [32768/60000 (55%)]	Loss: 1.578563

Test set: Average loss: 0.5589, Accuracy: 8689/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.955974
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.594646

Test set: Average loss: 0.2878, Accuracy: 9211/10000 (92%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.262533
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.225729

Test set: Average loss: 0.2110, Accuracy: 9421/10000 (94%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.317929
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.187613

Test set: Average loss: 0.1654, Accuracy: 9542/10000 (95%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.293806
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.219996

Test set: Average loss: 0.1394, Accuracy: 9609/10000 (96%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.355466
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.074005

Test set: Average loss: 0.1215, Accuracy: 9638/10000 (96%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.251741
Train Epoch: 7 [327

In [12]:
### Dropout of 0.75 from first convolution
model_75 = DrO_Net(prob=0.75).to(device)
optimizer = optim.SGD(model_75.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_75, device, train_loader, optimizer, epoch)
    test(model_75, device, test_loader)

if (save_model):
    torch.save(model_75.state_dict(),"SavedWeights/mnist_cnn_dr_75.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.319533
Train Epoch: 1 [32768/60000 (55%)]	Loss: 1.514200

Test set: Average loss: 0.5469, Accuracy: 8742/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.697336
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.586879

Test set: Average loss: 0.3003, Accuracy: 9191/10000 (92%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.361754
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.286361

Test set: Average loss: 0.2224, Accuracy: 9375/10000 (94%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.355320
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.240517

Test set: Average loss: 0.1792, Accuracy: 9489/10000 (95%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.277045
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.148382

Test set: Average loss: 0.1531, Accuracy: 9551/10000 (96%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.177687
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.281524

Test set: Average loss: 0.1352, Accuracy: 9585/10000 (96%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.186457
Train Epoch: 7 [327

In [13]:
### Dropout of 11 from first convolution
model_100 = DrO_Net(prob=1).to(device)
optimizer = optim.SGD(model_100.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_100, device, train_loader, optimizer, epoch)
    test(model_100, device, test_loader)

if (save_model):
    torch.save(model_100.state_dict(),"SavedWeights/mnist_cnn_dr_100.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303498
Train Epoch: 1 [32768/60000 (55%)]	Loss: 2.300954

Test set: Average loss: 2.3100, Accuracy: 1149/10000 (11%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.298736
Train Epoch: 2 [32768/60000 (55%)]	Loss: 2.296218

Test set: Average loss: 2.3097, Accuracy: 1148/10000 (11%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.303009
Train Epoch: 3 [32768/60000 (55%)]	Loss: 2.308222

Test set: Average loss: 2.3095, Accuracy: 1147/10000 (11%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.304306
Train Epoch: 4 [32768/60000 (55%)]	Loss: 2.300113

Test set: Average loss: 2.3093, Accuracy: 1241/10000 (12%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 2.294951
Train Epoch: 5 [32768/60000 (55%)]	Loss: 2.304523

Test set: Average loss: 2.3091, Accuracy: 1291/10000 (13%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 2.299289
Train Epoch: 6 [32768/60000 (55%)]	Loss: 2.299188

Test set: Average loss: 2.3090, Accuracy: 1340/10000 (13%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 2.309175
Train Epoch: 7 [327

Best among all the above dropout settings, the architecturure with 50% dropout performed the best. As expected, a dropout of 1 is messing up the network. 

#### C
Including Batch Normalization to the architecture and testing the results


In [14]:
class Best_Net(nn.Module):
    def __init__(self, prob=0.5):
        super(Best_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.do1 = nn.Dropout(prob)
        self.bn1 = torch.nn.BatchNorm2d(20)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.do1(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
class Best_Net_BN(nn.Module):
    def __init__(self):
        super(Best_Net_BN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.bn1 = torch.nn.BatchNorm2d(20)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [15]:
### Dropout and BatchNorm model
model_DrBn = Best_Net().to(device)
optimizer = optim.SGD(model_DrBn.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_DrBn, device, train_loader, optimizer, epoch)
    test(model_DrBn, device, test_loader)

if (save_model):
    torch.save(model_DrBn.state_dict(),"SavedWeights/mnist_cnn_drbn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302711
Train Epoch: 1 [32768/60000 (55%)]	Loss: 1.154164

Test set: Average loss: 0.4399, Accuracy: 9004/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.559795
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.459656

Test set: Average loss: 0.2303, Accuracy: 9377/10000 (94%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.241235
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.261564

Test set: Average loss: 0.1683, Accuracy: 9542/10000 (95%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.128333
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.152910

Test set: Average loss: 0.1351, Accuracy: 9606/10000 (96%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.271201
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.075260

Test set: Average loss: 0.1146, Accuracy: 9661/10000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.129935
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.169259

Test set: Average loss: 0.1003, Accuracy: 9688/10000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.160761
Train Epoch: 7 [327

In [16]:
### Just BatchNorm model
model_Bn = Best_Net_BN().to(device)
optimizer = optim.SGD(model_Bn.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_Bn, device, train_loader, optimizer, epoch)
    test(model_Bn, device, test_loader)

if (save_model):
    torch.save(model_Bn.state_dict(),"SavedWeights/mnist_cnn_bn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309135
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.794010

Test set: Average loss: 0.3387, Accuracy: 9204/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.335519
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.294478

Test set: Average loss: 0.1951, Accuracy: 9459/10000 (95%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.151626
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.119101

Test set: Average loss: 0.1512, Accuracy: 9573/10000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.110375
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.229766

Test set: Average loss: 0.1262, Accuracy: 9622/10000 (96%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.162882
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.180243

Test set: Average loss: 0.1059, Accuracy: 9679/10000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.108464
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.119780

Test set: Average loss: 0.0950, Accuracy: 9696/10000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.106597
Train Epoch: 7 [327

### Observations

From the above train test prediction results, it is pretty clear that the network with a dropout of 50% and a batch normalization layer right after convolutional output is the best configuration for this task. It outperformal all the previous discussed configurations.

### D

In [17]:
class BN_Xavier(nn.Module):
    def __init__(self):
        super(BN_Xavier, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.bn1 = torch.nn.BatchNorm2d(20)
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


class BN_Kamming(nn.Module):
    def __init__(self):
        super(BN_Kamming, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.bn1 = torch.nn.BatchNorm2d(20)
        nn.init.kaiming_uniform_(self.conv1.weight)
        nn.init.kaiming_uniform_(self.conv2.weight)
        nn.init.kaiming_uniform_(self.fc1.weight)
        nn.init.kaiming_uniform_(self.fc2.weight)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [18]:
### Xavier init model
model_Xavier = BN_Xavier().to(device)
optimizer = optim.SGD(model_Xavier.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_Xavier, device, train_loader, optimizer, epoch)
    test(model_Xavier, device, test_loader)

if (save_model):
    torch.save(model_Xavier.state_dict(),"SavedWeights/mnist_cnn_xavier.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.939128
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.414906

Test set: Average loss: 0.1933, Accuracy: 9493/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.371297
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.092396

Test set: Average loss: 0.1250, Accuracy: 9650/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.189232
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.028351

Test set: Average loss: 0.0989, Accuracy: 9714/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.049379
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.123973

Test set: Average loss: 0.0848, Accuracy: 9745/10000 (97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.132394
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.078615

Test set: Average loss: 0.0746, Accuracy: 9775/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.052723
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.081344

Test set: Average loss: 0.0674, Accuracy: 9788/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.030365
Train Epoch: 7 [327

In [19]:
### Kaiming init model
model_Kaiming = BN_Kamming().to(device)
optimizer = optim.SGD(model_Kaiming.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_Kaiming, device, train_loader, optimizer, epoch)
    test(model_Kaiming, device, test_loader)

if (save_model):
    torch.save(model_Kaiming.state_dict(),"SavedWeights/mnist_cnn_kaiming.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.922023
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.276075

Test set: Average loss: 0.2233, Accuracy: 9379/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.261895
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.288560

Test set: Average loss: 0.1507, Accuracy: 9571/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.080885
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.237646

Test set: Average loss: 0.1232, Accuracy: 9660/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.126425
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.072083

Test set: Average loss: 0.1032, Accuracy: 9698/10000 (97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.248558
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.067943

Test set: Average loss: 0.0910, Accuracy: 9733/10000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.096007
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.098278

Test set: Average loss: 0.0830, Accuracy: 9749/10000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.106347
Train Epoch: 7 [327

In [20]:
### Basic architecture with Xavier initialization, no BatchNorm

class TmpXavier(nn.Module):
    def __init__(self):
        super(TmpXavier, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [21]:
### Basic Xavier init model
model_XavTEmp = TmpXavier().to(device)
optimizer = optim.SGD(model_XavTEmp.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_XavTEmp, device, train_loader, optimizer, epoch)
    test(model_XavTEmp, device, test_loader)

if (save_model):
    torch.save(model_XavTEmp.state_dict(),"SavedWeights/mnist_cnn_XavTemp.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.347324
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.692414

Test set: Average loss: 0.2906, Accuracy: 9160/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.312031
Train Epoch: 2 [32768/60000 (55%)]	Loss: 0.332909

Test set: Average loss: 0.1933, Accuracy: 9433/10000 (94%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.231090
Train Epoch: 3 [32768/60000 (55%)]	Loss: 0.109227

Test set: Average loss: 0.1549, Accuracy: 9545/10000 (95%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.122124
Train Epoch: 4 [32768/60000 (55%)]	Loss: 0.197828

Test set: Average loss: 0.1262, Accuracy: 9623/10000 (96%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.132507
Train Epoch: 5 [32768/60000 (55%)]	Loss: 0.072143

Test set: Average loss: 0.1102, Accuracy: 9663/10000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.063120
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.154778

Test set: Average loss: 0.0944, Accuracy: 9743/10000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.059600
Train Epoch: 7 [327

From the above few cells we can conclude the following

1. BatchNormalisation speeds up training. The hidden layers are now receiving a lesser deviation in input if batch normalisation operation is done. 

2. A good initialisation can always save time for some particular tasks. In our case, Xavier + BN gave the best result so far within 20 epochs

3. Batch Normalisation reduced the gap between the convergence point of networks after 20 epochs. Roughly all networks gave more than 9830 images correctly classified